In [19]:
from sentence_transformers import SentenceTransformer
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ast
import plotly

import json

from dotenv import load_dotenv
import os
import requests

## Variables de entorno

In [3]:
# Cargar variables de entorno
load_dotenv()

# Leer la API Key
api_key = os.getenv("GROQ_API_KEY")


## Funciones

In [4]:
def safe_parse_conocimientos(value):
    '''
    Cambia de formato las entradas de la forma
    "'palabra_1','palabra_2',..., 'palabra_n'"
    a
    ['palabra_1','palabra_2,...,'palabra_n']
    '''
    try:
        # Si no está entre corchetes, lo envolvemos
        if not value.strip().startswith("["):
            value = f"[{value}]"
        return ast.literal_eval(value)
    except Exception:
        return [str(value)]  # Si no se puede evaluar, lo devolvemos como lista con un solo string

In [6]:
def get_cluster_cardinality(linked_matrix, num_clusters):
    """
    Given a linkage matrix and the desired number of clusters, 
    this function returns the cardinality (sizes) of each cluster.
    
    Parameters:
    linked_matrix (ndarray): The linkage matrix from hierarchical clustering.
    num_clusters (int): The number of clusters to form.

    Returns:
    unique_clusters (ndarray): Unique cluster labels.
    cluster_counts (ndarray): Cardinality of each cluster (number of points in each cluster).
    """
    # Use fcluster to get cluster labels (1, 2, ..., num_clusters)
    cluster_labels = fcluster(linked_matrix, num_clusters, criterion='maxclust')

    # Get the unique cluster labels and their cardinality (size)
    unique_clusters, cluster_counts = np.unique(cluster_labels, return_counts=True)
    
    return unique_clusters, cluster_counts

**TO DO:**
1. Definir cual valor para el parámetro "criterion" de la función **fcluster**: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html
2. Identificar cual es la mejor linkage function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html
3. Identificar las métricas para evaluar el ejercicio

## Lectura datos

In [7]:
conoc_df=pd.read_excel("../data/conoc_tej.xlsx",index_col=False)
conoc_df=conoc_df.replace('NA',np.nan)
conoc_df.shape

(3087, 26)

In [8]:
#Conocimientos de la vacante
conoc_df['conocimientos'][0]

"'mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica'"

## Clusterizarion semántica

**Pipeline:**
1. Ajustar corpus
2. Instanciar embedding
3. Computar el hierarchical clustering:
    >linkage functions
    >- average
    >-  single
    >- ward
    
    >Clusters in dendogram: 30 -> 10


In [9]:
#1. Define the corpus
corpus = conoc_df['conocimientos'].apply(safe_parse_conocimientos).sum()
print(f"Numero de documentos en el corpus: {len(corpus)}")

Numero de documentos en el corpus: 16508


In [10]:
# 2. Embed skills
model = SentenceTransformer('all-MiniLM-L6-v2')
skill_embeddings = model.encode(corpus)

print(skill_embeddings.shape)

(16508, 384)


In [11]:
# 3.1 Compute hierarchical clustering
#using avg metric
linked_avg = linkage(skill_embeddings, method='average')  # average distance between points
unique_clusters_avg, cluster_counts_avg = get_cluster_cardinality(linked_avg, 30)

In [12]:
# 3.2 Compute hierarchical clustering
#using single metric
linked_single = linkage(skill_embeddings, method='single')  # nearest point
unique_clusters_single, cluster_counts_single = get_cluster_cardinality(linked_single, 30)

In [13]:
# 3.3 Compute hierarchical clustering
#using ward metric
linked_ward = linkage(skill_embeddings, method='ward')  # "ward" tries to minimize variance
unique_clusters_ward, cluster_counts_ward = get_cluster_cardinality(linked_ward, 30)


In [15]:
# Print the results
print("Number of clusters - Avg Metric:", len(unique_clusters_avg))
print("Cardinality:", cluster_counts_avg)
print("")

print("Number of clusters - Single Metric:", len(unique_clusters_single))
print("Cardinality:", cluster_counts_single)
print("")

print("Number of clusters - Ward Metric:", len(unique_clusters_ward))
print("Cardinality:", cluster_counts_ward)

Number of clusters - Avg Metric: 30
Cardinality: [    1     1     4     8     6     3     3    40    14     1    35   112
    37     3    20     6    90   390     7     5    30   710    50     7
   316   185   118 14290     2    14]

Number of clusters - Single Metric: 30
Cardinality: [    4     2     2     2     2     8     2     2     3     2     2    10
     2     4     2 16445     1     1     1     1     1     1     1     1
     1     1     1     1     1     1]

Number of clusters - Ward Metric: 30
Cardinality: [ 250  575  331  235  158 1031  140 1046  163  140  192  447  324  295
  270  458  485 2311 1583 1601   70  154  254 2681   83  253  142  249
  320  267]


In [16]:
# 5. Cut the dendrogram into "k" clusters
num_clusters = 10
cluster_labels = fcluster(linked_ward, num_clusters, criterion='maxclust')

In [17]:
# 6. Print results
n=1
for skill, cluster_id in zip(corpus, cluster_labels):
    if n<=10:
        print(f"Skill: {skill}, Cluster: {cluster_id}")
        n+=1
    else: 
        break

Skill: mantenimiento de maquinaria industrial, Cluster: 7
Skill: manejo de bombas de vacío, Cluster: 7
Skill: equipos de soldadura, Cluster: 7
Skill: mecánica hidráulica, Cluster: 7
Skill: Control integrado de plagas, Cluster: 6
Skill: Control de roedores, Cluster: 6
Skill: Desinsectación, Cluster: 7
Skill: Lavado y desinfección de tanques, Cluster: 7
Skill: Curso de trabajo en alturas, Cluster: 5
Skill: Licencias A2 y C1, Cluster: 7


In [18]:
#Ver una muestra de las habilidades en un cluster 
from collections import defaultdict

cluster_skills = defaultdict(list)
for skill, cluster_id in zip(corpus, cluster_labels):
    cluster_skills[cluster_id].append(skill)

for cluster_id, cluster_list in cluster_skills.items():
    #truncar el print
    print(f"Cluster {cluster_id}: {cluster_list[0:30]}")
    break

Cluster 7: ['mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica', 'Desinsectación', 'Lavado y desinfección de tanques', 'Licencias A2 y C1', 'Inglés', 'Gestora y consultora asesora', 'NO_APLICA', 'Mercado de energía colombiano', 'Cobro de cartera', 'Manejo de tecnicos', 'Medida directa y semidirecta', 'Construcción', 'Manejo de correspondencia', 'Regulaciones gubernamentales', 'Herramienta SAP', 'Planeación de compras y contratación', 'Herramienta SAP', 'NO_APLICA', 'NO_APLICA', 'Equipos de calentamiento', 'Sistema de aire a presión', 'Consumo de agua', 'Huella de carbono', 'Mejora continua', 'Limpieza y lubricación de máquinas y equipos', 'Mantenimientos preventivos de maquinaria', 'Lavados de techos de casetas']


In [21]:
# Convert keys to int
cluster_skills_serializable = {int(k): v for k, v in cluster_skills.items()}
#guardar el resultado de la clusterizacion
with open('../output/cluster_skills.json', 'w') as f:
    json.dump(dict(cluster_skills_serializable), f)

## Etiquetar el cluster con un LLM

In [38]:
#Por el momento se etiqueta un cluster a partir de 500 elementos del cluster (numero arbitrario para que la api no entre en timeout)

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

#E.g Primer cluster
skills_list = cluster_skills[2][:500]

# Spanish prompt
prompt = f"Dado el siguiente listado de habilidades/conocimientos laborales: {', '.join(skills_list)}, retorna nombres de max 10 palabras que resuma las habilidades"

payload = {
    "model": "llama3-8b-8192",  # Or "llama3-70b-8192"
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

response = requests.post("https://api.groq.com/openai/v1/chat/completions", json=payload, headers=headers)

# Verificamos si todo salió bien
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

Error 429: {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jsskhw9de0ba48xr8d0ekbds` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 2144, Requested 4225. Please try again in 3.685s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

